In [1]:
import os
CURDIR = os.getcwd().split("\\")
if "notebooks" in CURDIR:
    n = len(CURDIR) - CURDIR.index("notebooks")
    os.chdir(os.getcwd() + ("\\.." * n))

In [2]:
import seaborn as sns
import polars as pl
diamonds = pl.DataFrame(sns.load_dataset("diamonds"))

In [ ]:
def prep_pivot_table(df, index, outcomes, agg_fun):
    

In [100]:
# Pivot Table:
pivot_tab = diamonds.pivot(values="price", index="cut", columns="color", aggregate_function=pl.element().mean())
# Convert to long format:
pivot_tab.melt(id_vars="cut")

cut,variable,value
cat,str,f64
"""Ideal""","""E""",2597.55009
"""Premium""","""E""",3538.91442
"""Good""","""E""",3423.644159
"""Very Good""","""E""",3214.652083
"""Fair""","""E""",3682.3125
"""Ideal""","""I""",4451.970377
"""Premium""","""I""",5946.180672
"""Good""","""I""",5078.532567
"""Very Good""","""I""",5255.879568


In [9]:
from prep.prep_freq_table import prep_freq_table
prep_freq_table(sns.load_dataset("diamonds"), "cut", report=True, digits=1)

,cut,count,proportion
0,Good,4906,9.1%
1,Premium,13791,25.6%
2,Fair,1610,3.0%
3,Ideal,21551,40.0%
4,Very Good,12082,22.4%


In [41]:
diamonds.select("cut") # one variable
diamonds.select(["cut","color"]) # multiple variables

# Select based on regular expression
def select_if(df, regex=r"."):
    import re
    return df.select([col for col in df.columns if bool(re.search(regex,col))])

#select_if(diamonds, r"^c") 
diamonds.pipe(select_if,r"^c")

carat,cut,color,clarity
f64,cat,cat,cat
0.23,"""Ideal""","""E""","""SI2"""
0.21,"""Premium""","""E""","""SI1"""
0.23,"""Good""","""E""","""VS1"""
0.29,"""Premium""","""I""","""VS2"""
0.31,"""Good""","""J""","""SI2"""
0.24,"""Very Good""","""J""","""VVS2"""
0.24,"""Very Good""","""I""","""VVS1"""
0.26,"""Very Good""","""H""","""SI1"""
0.22,"""Fair""","""E""","""VS2"""


In [13]:
for col in diamonds.columns:
    print(bool(re.search(r"c",col)))

True
True
True
True
False
False
True
False
False
False


In [56]:
# Spezifische Auswahl
diamonds.filter(
    (pl.col("cut") == "Fair") | (pl.col("cut") == "Ideal")
)

# Filtern via Regex:
diamonds.filter(
    pl.col("cut").cast(str).str.contains("Fair|Ideal") 
)

# Is Not in
diamonds.filter(
    ~pl.col("cut").cast(str).is_in(["Fair", "Ideal"])
)
# Is in:
diamonds.filter(
    pl.col("cut").cast(str).is_in(["Fair", "Ideal"])
)

carat,cut,color,clarity,depth,table,price,x,y,z
f64,cat,cat,cat,f64,f64,i64,f64,f64,f64
0.23,"""Ideal""","""E""","""SI2""",61.5,55.0,326,3.95,3.98,2.43
0.22,"""Fair""","""E""","""VS2""",65.1,61.0,337,3.87,3.78,2.49
0.23,"""Ideal""","""J""","""VS1""",62.8,56.0,340,3.93,3.9,2.46
0.31,"""Ideal""","""J""","""SI2""",62.2,54.0,344,4.35,4.37,2.71
0.3,"""Ideal""","""I""","""SI2""",62.0,54.0,348,4.31,4.34,2.68
0.33,"""Ideal""","""I""","""SI2""",61.8,55.0,403,4.49,4.51,2.78
0.33,"""Ideal""","""I""","""SI2""",61.2,56.0,403,4.49,4.5,2.75
0.33,"""Ideal""","""J""","""SI1""",61.1,56.0,403,4.49,4.55,2.76
0.23,"""Ideal""","""G""","""VS1""",61.9,54.0,404,3.93,3.95,2.44


In [73]:
diamonds.groupby("cut").agg(
    pl.col("price").apply(lambda s : {"min":s.min(), "max":s.max()})
).unnest("price")

cut,max,min
cat,i64,i64
"""Ideal""",18806,326
"""Premium""",18823,326
"""Very Good""",18818,336
"""Good""",18788,327
"""Fair""",18574,337


In [87]:
diamonds.groupby("cut").agg(
    pl.exclude(pl.Utf8, pl.Categorical).mean().prefix("mean_"), # Mean of all numeric Variables,
    pl.col(pl.Float64, pl.Int64).sum().prefix("sum_") # Sum of all numeric Variables
)

cut,mean_carat,mean_depth,mean_table,mean_price,mean_x,mean_y,mean_z,sum_carat,sum_depth,sum_table,sum_price,sum_x,sum_y,sum_z
cat,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64
"""Premium""",0.891955,61.264673,58.746095,4584.257704,5.973887,5.944879,3.647124,12300.95,844901.1,810167.4,63221498,82385.88,81985.82,50297.49
"""Ideal""",0.702837,61.709401,55.951668,3457.54197,5.507451,5.52008,3.401448,15146.84,1.3299e6,1.2058e6,74513487,118691.07,118963.24,73304.61
"""Fair""",1.046137,64.041677,59.053789,4358.757764,6.246894,6.182652,3.98277,1684.28,103107.1,95076.6,7017600,10057.5,9954.07,6412.26
"""Very Good""",0.806381,61.818275,57.95615,3981.759891,5.740696,5.770026,3.559801,9742.7,746888.4,700226.2,48107623,69359.09,69713.45,43009.52
"""Good""",0.849185,62.365879,58.694639,3928.864452,5.838785,5.850744,3.639507,4166.1,305967.0,287955.9,19275009,28645.08,28703.75,17855.42


In [75]:
diamonds.groupby("cut").agg(
    pl.col("price").apply(lambda s : [s.min(),s.max()])
).explode("price")

cut,price
cat,i64
"""Ideal""",326
"""Ideal""",18806
"""Premium""",326
"""Premium""",18823
"""Very Good""",336
"""Very Good""",18818
"""Good""",327
"""Good""",18788
"""Fair""",337


In [59]:
prep_table = data.groupby(group_vars[1:]).agg(
            pl.col(group_vars[0]).apply(lambda s: s.value_counts().to_dict())
        ).unnest(group_vars[0]).explode(["","counts"]).rename(
            {"": group_vars[0]}
        )

carat,cut,color,clarity,depth,table,price,x,y,z
f64,cat,cat,cat,f64,f64,i64,f64,f64,f64
0.21,"""Premium""","""E""","""SI1""",59.8,61.0,326,3.89,3.84,2.31
0.23,"""Good""","""E""","""VS1""",56.9,65.0,327,4.05,4.07,2.31
0.29,"""Premium""","""I""","""VS2""",62.4,58.0,334,4.2,4.23,2.63
0.31,"""Good""","""J""","""SI2""",63.3,58.0,335,4.34,4.35,2.75
0.24,"""Very Good""","""J""","""VVS2""",62.8,57.0,336,3.94,3.96,2.48
0.24,"""Very Good""","""I""","""VVS1""",62.3,57.0,336,3.95,3.98,2.47
0.26,"""Very Good""","""H""","""SI1""",61.9,55.0,337,4.07,4.11,2.53
0.23,"""Very Good""","""H""","""VS1""",59.4,61.0,338,4.0,4.05,2.39
0.3,"""Good""","""J""","""SI1""",64.0,55.0,339,4.25,4.28,2.73
